In [303]:
import pandas as pd
import numpy as np
import re
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from collections import Counter
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import FeatureUnion, Pipeline

In [304]:
def nums_in_str(s):
    l = len(s)
    integ = []
    i = 0
    while i < l:
        s_int = ''
        a = s[i]
        while '0' <= a <= '9':
            s_int += a
            i += 1
            if i < l:
                a = s[i]
            else:
                break
        i += 1
        if s_int != '':
            integ.append(int(s_int))
    return integ

#nums_in_str('Время107 мин. / 01:47')

In [305]:
df = pd.read_csv('films.csv', ';', encoding='cp1251')
df

,year,country,genre,director,scripter,producer,premier_data,age_bound,duration,title,actors,rating
0,Год производства2021,"СтранаСША, Канада, Япония","Жанрфантастика, боевик, комедияслова",РежиссерШон Леви,"СценарийМэтт Либерман, Зак Пенн","ПродюсерГрег Берланти, Адам Колбеннер, Сара Ше...","Премьера в мире10 августа 2021, ...",Возраст16+,Время115 мин. / 01:55,Free Guy,Райан РейнольдсДжоди КомерТайка ВайтитиКамилль...,7.6
1,Год производства2021,"СтранаСША, Канада, Великобритания","Жанрбоевик, фантастика, комедия, фэнтезислова",РежиссерДжеймс Ганн,СценарийДжеймс Ганн,"ПродюсерАриэнн Бенедетти, Уолтер Хамада, Саймо...","Премьера в мире28 июля 2021, ...",Возраст18+,Время132 мин. / 02:12,The Suicide Squad,Идрис ЭльбаМарго РоббиДжон СинаЮэль КиннаманДа...,7.0
2,Год производства2021,"СтранаСША, Сербия","Жанружасы, триллерслова",РежиссерРодольфо Саягес,"СценарийФедерико Альварес, Родольфо Саягес","ПродюсерФедерико Альварес, Сэм Рэйми, Роб Тапе...","Премьера в мире12 августа 2021, ...",Возраст18+,Время98 мин. / 01:38,Don't Breathe 2,Стивен ЛэнгБрендан Секстон IIIМэдлин ГрэйсСтеф...,6.5
3,Год производства2021,СтранаСША,"Жанртриллер, драма, детективслова",РежиссерМ. Найт Шьямалан,"СценарийМ. Найт Шьямалан, Пьер Леви, Фредерик ...","ПродюсерМарк Бьенсток, Ашвин Раджан, Стивен Шн...","Премьера в мире21 июля 2021, ...",Возраст16+,Время113 мин. / 01:53,Old,Гаэль Гарсиа БернальВики КрипсРуфус СьюэллКен ...,6.3
4,Год производства2021,СтранаСША,"Жанрприключения, комедия, фэнтезислова",РежиссерЖауме Кольет-Серра,"СценарийГленн Фикарра, Джон Рекуа, Майкл Грин,...","ПродюсерДжон Дэвис, Бо Флинн, Джон Фокс, ...","Премьера в мире24 июля 2021, ...",Возраст12+,Время127 мин. / 02:07,Jungle Cruise,Эмили БлантДуэйн ДжонсонЭдгар РамиресДжек Уайт...,6.6
5,Год производства2021,СтранаРоссия,Жанрдрама,"РежиссерМаксим Бриус, Михаил Вассербаум",СценарийВладимир Измайлов,"ПродюсерСергей Щеглов, Александр Колдаков",NaN,Возраст18+,Время-,18+,Александр БухаровМарина ДенисоваГлеб БорисовМа...,–
6,Год производства2021,СтранаСША,"Жанрмультфильм, комедия, приключения, семейный...",РежиссерТом МакГрат,"СценарийМайкл МакКаллерс, Марла Фрэйзи, Том Ма...","ПродюсерДжефф Хэрман, Том МакГрат, Джон Эрик Ш...","Премьера в мире1 июля 2021, ...",Возраст6+,Время107 мин. / 01:47,The Boss Baby: Family Business,Алек БолдуинДжеймс МарсденЭми СедарисАриана Гр...,5.6
7,Год производства2021 (1 сезон),СтранаРоссия,"Жанрдрама, триллер, фантастика",РежиссерАндрей Прошкин,"СценарийАлександр Лунгин, Роман Волобуев, Елен...","ПродюсерАлександр Плотников, Софья Митрофанова...",NaN,Возраст18+,Время48 мин.,18+,Алексей ФилимоновАртур СмольяниновДарья Савель...,–
8,Год производства2021,"СтранаФранция, Мексика, США, Швейцария, Бельги...","Жанрмюзикл, драма, мелодрамаслова",РежиссерЛеос Каракс,"СценарийРон Мэл, Расселл Мэл","ПродюсерШарль Жилибер, Пол-Доминик Уин Вакараз...","Премьера в мире6 июля 2021, ...",Возраст18+,Время141 мин. / 02:21,Annette,Адам ДрайверМарион КотийярСаймон ХелбергДевин ...,7.5
9,Год производства2021,СтранаСША,"Жанрфантастика, боевик, приключенияслова",РежиссерКейт Шортланд,"СценарийЭрик Пирсон, Жаклин Шеффер, Нед Бенсон","ПродюсерЗакария Алауи, Виктория Алонсо, Митчел...","Премьера в мире30 июня 2021, ...",Возраст16+,Время134 мин. / 02:14,Black Widow,Скарлетт ЙоханссонФлоренс ПьюДэвид ХарборО. Т....,6.2


In [306]:
class ValueCleanerDeleterTransformer(TransformerMixin):
    def __init__(self):
        return None
    
    def fit(self, X, y=None):

        return self
    
    def transform(self, X, y=None):
        df_ = X.copy()
        df_['year'] = [str(s)[-4::] for s in df_['year']]
        df_['country'] = [s.replace('Страна', '') for s in df_['country']]
        df_['genre'] = [s[4::].replace('слова', '') for s in df_['genre']]
        df_['director'] = [s.replace('Режиссер', '') for s in df_['director']]
        df_['scripter'] = [s.replace('Сценарий', '') for s in df_['scripter']]
        df_['producer'] = [s.replace('Продюсер', '') for s in df_['producer']]
        #df['premier_data'] = [str(s)[15:-5:] for s in df['premier_data']]
        df_['age_bound'] = [str(s).replace('Возраст', '') for s in df_['age_bound']]
        
        for index, row in df_.iterrows():
            if not row['year'].isdigit():
                df_.drop(index, inplace=True)
        for index, row in df_.iterrows():
            if len(nums_in_str(row['duration'])) > 0:
                row['duration'] = nums_in_str(row['duration'])[0]
            else:
                row['duration'] = '-'
            for index, row in df_.iterrows():
                row['scripter'] = row['scripter'].replace(', ...', '')

        for index, row in df_.iterrows():      
            row['rating'] = row['rating'].replace('%', '')
            if row['rating'] != '–':
                if float(row['rating']) > 10:
                    row['rating'] = float(row['rating'])/10

        df_ = df_.loc[~df_['title'].isin(['18+', '16+', '6+', '12+'])] # ~ means 'not'

        for index, row in df_.iterrows():
            i = 1
            while i != len(row['actors']):
                if row['actors'][i-1].islower() and row['actors'][i].isupper() or row['actors'][i-1] == 'I' and row['actors'][i] != 'I' and row['actors'][i].isupper():
                    row['actors'] = row['actors'][:i] + ', ' + row['actors'][i:]
                i += 1
        del df_['producer']
        del df_['premier_data']
        del df_['scripter']
        del df_['director']
        del df_['actors']
        
        return df_

In [307]:
class ImputeSymbolsByMean(TransformerMixin):
    
    def __init__(self,missing_symbol=np.nan, rnd=False):
        self.miss_symbol = missing_symbol
        self.rnd = rnd
        self.imputer = SimpleImputer()
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        print(len(X))
        X_copy = X.apply(lambda x : [np.nan if xi == self.miss_symbol else xi for xi in x])
        print(len(X_copy))
        X_copy = self.imputer.fit_transform(np.array(X_copy).reshape(-1,1))
        return X_copy

In [308]:
#fs = FeatureSelector(['duration'])
#cleaner = ValueCleanerDeleterTransformer()
#imp = ImputeSymbolsByMean(name='duration', missing_symbol='-')
#imp.fit_transform(fs.fit_transform(cleaner.fit_transform(df)))

In [309]:
class OneHotEncoderWithLabels(TransformerMixin):
    """
    Takes named Series, OHEs it and return Data Frame
    with names
    """
    def __init__(self):
        """
        Initalizes new instance of KNNInlineImputer
        """
        TransformerMixin.__init__(self)
        self.ohe = OneHotEncoder(sparse=False)
    
    def fit(self, X, y=None):
        """
        Fits to X by creating data frame to be returned
        """
        basic_name = X.columns[0]
        #basic_name = X.name
        values = sorted(list(set(X.columns[0])))
        self.res_df = pd.DataFrame(self.ohe.fit_transform(np.array(X.columns[0]).reshape(-1,1)))
        index = 0
        self.feature_names = values
        for col in self.res_df:
            self.res_df.rename(columns={col:str(basic_name + '_' + values[index])}, inplace=True)
            index += 1
        return self
        
    def transform(self, X, y=None):
        """
        Transforms X, but really returns Data Frame
        that created at 'fit' step
        """
        return self.res_df
    
    def get_feature_names(self):
        return self.feature_names

In [310]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    """
    The class provides basic functionality for retrieving
    a subset of columns from the dataset.
    """
    
    def __init__(self, feature_names):
        """
        Initialize class instance by setting
        a list of columns to retrieve from the dataset.
        """
        BaseEstimator.__init__(self)
        TransformerMixin.__init__(self)
        self.feature_names = feature_names
        
    def fit(self, X, y=None):
        """
        Fit FeatureSelector to X, but really do nothing.
        Return self.
        """
        return self
    
    def transform(self, X, y=None):
        """
        Transform X using feature selection. 
        Return column-subset of X.
        """
        print(self.feature_names)
        return X[self.feature_names]

In [311]:
class MultipleValuesOneHotEncoder(BaseEstimator, TransformerMixin):

    def __init__(self, drop):
        BaseEstimator.__init__(self)
        TransformerMixin.__init__(self)
        self.drop = drop
    
    def fit(self, X, y=None):
        return self
    
    
    def transform(self, X, y=None):
        X_copy = X.copy()
        columns = X_copy.columns
        new_columns = list()
        #print(columns)
        for col in columns:
            X_copy[col+'_'] = X_copy[col].apply(lambda x: str(x).split(', '))
            new_columns.append(col+'_')
            counter = Counter([item for sublist in X_copy[col+'_'].values for item in sublist])
            top_amount = len(counter)
            top_countries = counter.most_common(top_amount)
            other_countries = list(counter.keys())[top_amount::]
            for key, value in top_countries:
                X_copy[col+'_'+key] = X_copy[col+'_'].apply(lambda x : 1 if key in x else 0)

            X_copy[col+'_other'] = X_copy[col+'_'].apply(lambda x : 1 if True in list(other_country in x for other_country in other_countries) else 0)
            
        if self.drop:
            X_copy.drop(columns, axis=1, inplace=True)
            X_copy.drop(new_columns, axis=1, inplace=True)
        return X_copy

In [275]:
df = df.dropna()

In [276]:
#ohe_labels = OneHotEncoderWithLabels()

In [283]:
country_genre_pipeline = Pipeline(
    steps=[
        ('country_genre_selector', FeatureSelector(['country', 'genre'])),
        ('country_genre_transformer', MultipleValuesOneHotEncoder(drop=True))
    ]
)

age_bounds_pipeline = Pipeline(
    steps=[
        ('age_bounds_selector', FeatureSelector(['age_bound'])),
        ('age_bounds_encoder', OneHotEncoderWithLabels())
    ]
)

duration_pipeline = Pipeline(
    steps=[
        ('duration_selector', FeatureSelector(['duration'])),
        ('duration_imputer', ImputeSymbolsByMean(missing_symbol='-', rnd=True))
    ]
)

rating_pipeline = Pipeline(
    steps=[
        ('rating_selector', FeatureSelector(['rating'])),
        ('rating_imputer', ImputeSymbolsByMean(missing_symbol='-', rnd=True))
    ]
)

scaler_pipeline = Pipeline(
    steps=[
        ('scaler_selector', FeatureSelector(('rating', 'duration', 'year'))),
        ('scaler', MinMaxScaler())
    ]
)

In [295]:
union_pipeline = FeatureUnion(transformer_list=[
    ('country_genre', country_genre_pipeline),
    ('age_bounds', age_bounds_pipeline), 
    ('duration', duration_pipeline),
    ('rating', rating_pipeline)
])

In [296]:
full_pipeline = Pipeline(
        steps=[
            ('cleaner', ValueCleanerDeleterTransformer()), 
            ('union_pipeline', union_pipeline),
            ('scaler', scaler_pipeline),         
        ]
    )


In [312]:
cleaner = ValueCleanerDeleterTransformer()
s1 = cleaner.fit_transform(df)
s2 = country_genre_pipeline.fit_transform(s1)
s3 = pd.concat([s2, age_bounds_pipeline.fit_transform(s1)],axis=1)
s3

['country', 'genre']
['age_bound']


,country_США,country_Канада,country_Япония,country_Великобритания,country_Сербия,country_Франция,country_Мексика,country_Швейцария,country_Бельгия,country_Германия,...,genre_семейный,genre_криминал,genre_ужасы,genre_детектив,genre_мюзикл,genre_мелодрама,genre_аниме,genre_спорт,genre_other,age_bound__
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
1,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
2,1,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,NaN
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,NaN
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
6,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,NaN
8,1,0,1,0,0,1,1,1,1,1,...,0,0,0,0,1,1,0,0,0,NaN
9,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
10,1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,NaN
11,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,NaN
